<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/11_clustering_(example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# （実装例）日本語の単語ベクトルをクラスタリング

* 使用するデータは下記の場所にあるもの

 * https://github.com/shiroyagicorp/japanese-word2vec-model-builder

* Dask-MLのspectral clusteringでクラスタリングして、意味の近い単語が同じクラスタに属しているかをチェックする。

In [1]:
!if [ ! -e latest-ja-word2vec-gensim-model.zip ]; then wget http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip ; fi

--2023-07-07 15:02:00--  http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip
Resolving public.shiroyagi.s3.amazonaws.com (public.shiroyagi.s3.amazonaws.com)... 52.219.68.149, 52.219.68.161, 52.219.17.9, ...
Connecting to public.shiroyagi.s3.amazonaws.com (public.shiroyagi.s3.amazonaws.com)|52.219.68.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132936751 (127M) [application/zip]
Saving to: ‘latest-ja-word2vec-gensim-model.zip’

latest-ja-word2vec- 100%[===================>] 126.78M  42.1MB/s    in 3.0s    

2023-07-07 15:02:03 (42.1 MB/s) - ‘latest-ja-word2vec-gensim-model.zip’ saved [132936751/132936751]



In [2]:
!unzip -u latest-ja-word2vec-gensim-model.zip
#!unzip -u /content/drive/MyDrive/data/latest-ja-word2vec-gensim-model.zip

Archive:  latest-ja-word2vec-gensim-model.zip
  inflating: word2vec.gensim.model   
  inflating: word2vec.gensim.model.syn1neg.npy  
  inflating: word2vec.gensim.model.wv.syn0.npy  


In [3]:
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)

In [4]:
wv = model.wv

In [5]:
words = []
vectors = []
for word in wv.key_to_index.keys():
  words.append(word)
  vectors.append(wv.get_vector(word))

In [6]:
import numpy as np

words = np.array(words)
vectors = np.array(vectors)

## Dask-MLのspectral clustering

* Dask-MLについてはWebサイトを参照
 * https://ml.dask.org/

* scalableなspectral clustering（ただし近似が入る）が実装されているので、使ってみる。
 * https://ml.dask.org/clustering.html
* sklearnにもspectral clusteringはある。
 * https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html
 * こちらは近似が入らない代わりに、遅い。
* 参考
 * https://dask-ml-benchmarks.readthedocs.io/en/latest/auto_examples/plot_spectral_clustering.html

In [7]:
!pip install dask_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 4.3 MB/s eta 0:00:00


* 詳細を知りたい場合は下記のコードを読む。
 * https://github.com/dask/dask-ml/blob/main/dask_ml/cluster/spectral.py
* 下記の論文のFig. 2が上のコードで実装されていると思われる。
 * https://people.cs.umass.edu/~mahadeva/cs791bb/reading/fowlkes-nystrom.pdf

In [8]:
import dask_ml.cluster

In [9]:
n_clusters = 100
clusterer = dask_ml.cluster.SpectralClustering(
    n_clusters=n_clusters,
    persist_embedding=True,
    kmeans_params={"init":"k-means++"},
    random_state=123,
    )

In [10]:
%%time
clusterer.fit(vectors)

CPU times: user 3min 4s, sys: 12.7 s, total: 3min 17s
Wall time: 2min 32s


SpectralClustering(kmeans_params={'init': 'k-means++'}, n_clusters=100,
                   persist_embedding=True, random_state=123)

In [11]:
labels = clusterer.labels_.compute()

In [12]:
np.save('dask_sc_labels', labels)

In [13]:
labels

array([ 5, 30, 14, ...,  2,  1, 57], dtype=int32)

* クラスタのサイズ順にクラスタのインデックス（0~99）をソートする。

In [14]:
unique, counts = np.unique(labels, return_counts=True)
size_dict = dict(zip(unique, counts))
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

### クラスタのサイズを調べる

In [15]:
counts[sorted_clusters]

array([8204, 7208, 5701, 4965, 4895, 4778, 4718, 4713, 4708, 4659, 4646,
       4579, 4481, 4451, 4347, 4324, 4287, 4270, 4184, 4143, 4120, 4097,
       4074, 4017, 3976, 3860, 3847, 3783, 3730, 3729, 3690, 3662, 3592,
       3569, 3561, 3457, 3450, 3348, 3344, 3339, 3329, 3281, 3258, 3257,
       3250, 3232, 3231, 3226, 3221, 3210, 3162, 3125, 3118, 3094, 3086,
       3079, 3059, 3055, 3053, 3040, 3012, 2976, 2972, 2935, 2864, 2852,
       2848, 2829, 2828, 2781, 2781, 2770, 2741, 2727, 2680, 2679, 2671,
       2643, 2635, 2627, 2608, 2599, 2587, 2573, 2571, 2560, 2531, 2487,
       2459, 2349, 2348, 2307, 2189, 2143, 2097, 2054, 1939, 1369, 1321,
        662])

### サイズが最大のクラスタを調べる

In [16]:
' '.join(words[labels == sorted_clusters[0]])

'松平正容 ユニットコム 福山中学校 広島県立福山誠之館高等学校 公立大学法人首都大学東京 小田喜 山本精一 マインド・ゲーム ポルデノーネ無声映画祭 京都大学大学院法学研究科 東京都立墨田川高等学校 六神合体ゴッドマーズ サンダーマスク 飯島敏宏 スーパーロボット レッドバロン ワンサくん プラレス3四郎 機甲猟兵メロウリンク 忍者戦隊カクレンジャー 超獣戦隊ライブマン 無限増殖 魔界村 狩野常信 especially フランケンシュタインの逆襲 浮草 6120 シークレット・ドクトリン ユーシーカード株式会社 吉助 大阪府大阪市平野区 ~、 田中学 ガンパウダー プリンスオブウェールズ 宮城県警察部 マエストラーレ 東海艦隊 南海艦隊 シレ 昭洋 世界フィギュアスケート国別対抗戦 おいでよ どうぶつの森 33打点 熊姫 ザ・イロモネア 小林麻耶 世界の果てまで セブン・アークス 9370 3080 Tatiana ラジオどっとあい 白峯寺 狩野安信 武蔵 MUSASHI ヒューマ 新地駅 鬼武者 秋元玲奈 RSIP ACD 公証人法 尽誠学園 力弥 オオ とっくり ここな 片岡我童 大岸 さえぎっ むざむざ 久松静児 黒部の太陽 海と毒薬 Outrage 1246 のろう パワプロクンポケット ハンゲ スタジオジャイアンツ 王立宇宙軍 オネアミスの翼 スターデジオ フリクリ 福岡県立小倉高等学校 福岡4区 滋賀県立彦根東高等学校 三菱重工業株式会社 エンガノ 超時空世紀オーガス 超時空騎団サザンクロス 機甲創世記モスピーダ 大鉄人17 超時空要塞マクロスII -LOVERS AGAIN- たかべ ルイジ・コメンチーニ 一興 予防外交 沖風 梶岡定道 丸茂 日本リーバ 機動戦士ガンダム 戦場の絆 サラマンドラ "|-|||||||| Stéphane Florent 育英高等学校 雲雀丘 3720 1595 朝夜 十日市場 松本電鉄バス テアトロン 砂の女 定州市 ウルトラジャンプ 月刊電撃コミックガオ! 別冊ヤングマガジン 特警ウインスペクター 地獄先生ぬ〜べ〜 Grosso 五星戦隊ダイレンジャー 太陽の王子 ホルスの大冒険 子鹿物語 デュアルヘッド hydroxide 利章 川崎市市民ミュージアム gsi 1583 Rémy ベール湖 メトロポール・ド・リヨ

### サイズが最小のクラスタを調べる

In [17]:
' '.join(words[labels == sorted_clusters[99]])

'求めよ 連れ戻そ だろ しよ ご 戻ろ 盗も あろ 受け取ろ 抑えよ 登ろ けんど 戻そ 取り持と つこ させよ 押し通そ せよ 示そ 集めよ こんど 出そ 討と 高めよ 行なお 語ろ 納めよ 言お 渡そ 及ぼ でしょ 立と 抜こ 斬ろ やろ 付けよ 見よ 殺そ 収めよ 立ち去ろ 引き返そ 慰めよ 切ろ いえよ 返そ もらお ましょ 帰ろ 見極めよ 殴ろ 探ろ してやろ くれよ かかろ まいと さそ 奪お あげよ 聞こ 刺そ 買お 捕らえよ 送ろ 贈ろ 戦お ちろ 備えよ ぶろ 迎えよ ほしょ えんど 決めよ 伝えよ 知らせよ 遊ぼ かいきょ 受けよ かんす 出よ いこ おこ 立て直そ かわそ 渡ろ たろ 保と かけよ 与えよ 役立てよ 行お 得よ 逃げ出そ 助けよ 近づこ 合わせよ ていりゅうじょ 洗お みよ 始まろ 図ろ 彷徨 届けよ 減らそ 辞めよ なろ 立てよ られよ 起こそ 封じよ 結ぼ 調べよ 救お ごまかそ 避けよ 抱きしめよ 防ご 歌お 守ろ 上げよ 導こ 述べよ 割ろ 打と 死の いよ 見せよ よかろ 取り戻そ 売り出そ 変えよ 目指そ 逃げよ 乗ろ 言えよ 妨げよ 知ろ 乗り切ろ 作り上げよ 育てよ 入れよ 認めよ 及ぼそ 加えよ 作ろ しじょ 乗り込も 留めよ 尽くそ 働こ 続けよ 組み込も 潰そ 広げよ 挑も 探そ げんこ 進も 座ろ 掴も ぎゅ 見出そ つけよ 書こ 倒そ 見直そ 開こ 追い払お 乗っ取ろ 用いよ 譲ろ 行こ 追い出そ れよ 説こ 絶する 置こ づけよ 笑お 盛り上げよ 食お やめよ じゃろ できよ 狙お 止めよ 盛り込も ぶしゅ 葬ろ 通り過ぎよ 取ろ 始めよ 捨てよ 応えよ 逃れよ 補お 握ろ 進めよ 強めよ 当たろ 起きよ 壊そ 描こ 採ろ 貫こ 持と 使お 極めよ 抜け出そ 果して 消そ 会お きんじろ 出かけよ 召し出そ ぎじょ とうじょ 貸そ 触ろ げつよ 仕立てよ 覚えよ 明かそ 済ませよ 稼ご 去ろ 貰お 捕まえよ 滅ぼそ 逃がそ 移そ 確かめよ 消えよ つくろ むそ 退けよ 就こ 封じ込めよ 暴こ 躍起 応じよ 仕えよ 定めよ 支えよ 攻めよ 考えよ 話そ 攻め込も 引こ 掛けよ 飛び込も 動かそ 隠そ 伸ばそ 持ち出そ はかろ もたらそ 売り込も 裂こ 引き出そ 挙げよ 広めよ 取り上げよ 深めよ はも

### サイズが中間的なクラスタを調べる

In [18]:
' '.join(words[labels == sorted_clusters[49]])

'】 \u3000 文学部 自治医科大学 学 麹町中学校 和光大学 首都大学東京 修士課程 関西大学 社会学部 台湾大学 法律学科 専攻 単位取得退学 京都大学 研究会 日本の教育 府立一中 東京都立日比谷高等学校 高等工業学校 工業大学 基礎科学 紫綬褒章 東京都立新宿高等学校 日本工芸会 日本文芸家協会 日本ペンクラブ 日本脚本家連盟 京都嵯峨芸術大学 客員教授 東京都出身 慶應義塾大学 国文 卒 慶大 名誉教授 東京大学 理学部 数物 研究員 東京神学大学 学長 福岡県出身 情報学 大阪大学 国立新美術館 上智大学 文科 教養学部 同大 東京大学東洋文化研究所 立教大学 助教授 人間科学部 阪大 人間科学 東京大学先端科学技術研究センター 政策研究大学院大学 法政大学 特任教授 早稲田大学 研究所 学会 青山学院大学 県立 女子高等師範学校 広島大学 教育学部 男子寮 香川県出身 公益財団法人 文化振興 芸術文化 神奈川県横浜市 工学部 応用化学 福岡大学 研 歴任 高等師範学校 上海大学 中国人民政治協商会議全国委員会 宮城師範学校 宮城県図書館 日本体育大学 体育学部 功労賞 文化功労者 日本芸術院 東亜大学 京都府京都市 府立 文学研究科 芸術選奨新人賞 恩賜賞 プレップスクール 東大 岡山県出身 理工学部 フランス文学者 身体運動 行政学者 日本学士院 東京都生まれ 法学部 国際基督教大学 研究科 東京市政調査会 21世紀臨調 リトルシニア 大阪府出身 文化財研究所 財政学 経営学 東京藝術大学 日本現代音楽協会 総合科学 体育学 全日本スキー連盟 BFA 一橋大学 アミューズメントメディア総合学院 ロボット競技 照明デザイナー 常磐大学 地学 NHK放送文化研究所 大阪外国語大学 ）< 中央民族大学 ティーチングアシスタント 電気工学 サイモンフレーザー 西オーストラリア大学 内科学 明治大学 商学部 医員 帝京 信州大学 文理学部 芸術選奨 愛知県名古屋市 全日本軟式野球連盟 審判学校 国士舘大学 東海大学 筑波大学 工学 関西学院大学 福井県立藤島高等学校 経済学部 農林水産 経営学部 修道中学校 前橋中学校 陸軍歩兵学校 医学博士 九州大学 学芸大学 健康科学部 滋賀県出身 東京都立大学附属高等学校 山梨県出身 歴史教育者協議会 日本政治史 政

* 全てのクラスタについて、**ランダムな50語**を表示

In [19]:
np.random.seed(0)

for i in range(len(sorted_clusters)):
  members = words[labels == sorted_clusters[i]]
  np.random.shuffle(members)
  print(' '.join(members[:50]))
  print('-'*80)

倍する ろうそ しゅうり マドー 1348 AYUSE KOZUE 大変さ チャタヌーガ・ルックアウツ 長陽 EF210 機動戦士ガンダムF90 Schiller 黄衣の王 シーエン 文身 日本SF新人賞 student referred 9370 独立行政法人経済産業研究所 メトロポール・ド・リヨン 怪物王女 三年式機関銃 ゆでん シニャンガ 荒井チェリー 1971年製 自動プログラミング シグニュー 黒須克彦 ピリカ ドラゴン桜 煮しめ 2686 松平義行 スペランツァFC大阪高槻 ヴァイルティンゲン Tia 沙村広明 ∑ 造作材 バトルドッジボール シレ ケイト・ベッキンセイル ハッピーデイズ 硫酸鉛 スターリツァ 川添智久 Hussain 他人の顔
--------------------------------------------------------------------------------
ラインホルト・メスナー トムフール 東京カテドラル聖マリア大聖堂 ウルトリア 紀ノ国屋 かかりきり 徳吉 キージンガー コスチュームプレイ しるす 静寛院宮 Toonz リュドヴィク・ジュリ ルイ・ブラン こどものおもちゃ 猪苗代氏 NNN昼のニュース ピュジョール 職役 ヘッズ・アップ 帝國館 あゆみの箱 素股 フェルナンド・ゴンサレス ネク 農業気象 ミニラボ 象徴詩 菊野 circular nmi 56.5% 悪球打ち 河角廣 コクトー・ツインズ 乗り継が パタプスコ ヘナン・バラオン 白松 スタジオアリス 11才 エストレラ スクィンチ PartIII 城南FC groupAsk 鍵のかかる部屋 ストークス・モーター 魚見 74式特大型トラック
--------------------------------------------------------------------------------
アイ・キャン Rae ギャル曽根 ガイアの夜明け ギルバート・アンド・サリヴァン ネギま!? FF2 「夢を死なせるわけにいかない」公演 Another World スレイヤーズ ソ・ラ・ノ・ヲ・ト UDON MINMI 女の一生 リブレ出版 たいやき カイ・シデン ソフト・ロック ヴァニラウェア 四号機 ヴェリー シャンブロウ アニメ魂 チバユ

### `n_components`を増やしてみる
* これを増やすことで、近似の度合いを下げることができる。
 * その代わり、実行時間は長くなる。
* デフォルトの設定では`n_components=100`

In [20]:
clusterer_200 = dask_ml.cluster.SpectralClustering(
    n_clusters=n_clusters,
    persist_embedding=True,
    kmeans_params={"init":"k-means++"},
    n_components=200,
    random_state=123,
    )

In [21]:
%%time
clusterer_200.fit(vectors)

CPU times: user 6min 48s, sys: 24.6 s, total: 7min 12s
Wall time: 5min 53s


SpectralClustering(kmeans_params={'init': 'k-means++'}, n_clusters=100,
                   n_components=200, persist_embedding=True, random_state=123)

### 埋め込み空間でのクラスタの重心を取得する
* 元のデータは、spectral embeddingによって、低次元空間に埋め込まれている。
* spectral clusteringでは、このembeddingを、最後にクラスタリングしている。
* このembeddingを取得するために、自前のクラスを定義する。
 * embeddingを取得すれば、KMeansのときと同じように、重心に近い順に単語をソートできる。

In [22]:
from dask_ml.cluster import KMeans

In [23]:
class MyKMeans(KMeans):
  def __init__(self, **kwargs):
    self.X = None
    super().__init__(**kwargs)
  def fit(self, X, y=None):
    self.X = X
    super().fit(X, y=y)

In [24]:
n_clusters = 100

my_kmeans = MyKMeans(
    n_clusters=n_clusters,
    random_state=123,
    init="k-means++",
    )

my_clusterer = dask_ml.cluster.SpectralClustering(
    n_clusters=n_clusters,
    persist_embedding=True,
    assign_labels=my_kmeans,
    random_state=123,
    )

In [25]:
%%time
my_clusterer.fit(vectors)

CPU times: user 2min 53s, sys: 8.16 s, total: 3min 1s
Wall time: 2min 24s


SpectralClustering(assign_labels=MyKMeans(), n_clusters=100,
                   persist_embedding=True, random_state=123)

In [26]:
U2 = my_clusterer.assign_labels.X.compute()
labels = my_clusterer.labels_.compute()
centers = my_clusterer.assign_labels.cluster_centers_

In [27]:
U2.shape

(335476, 100)

In [28]:
centers.shape

(100, 100)

In [29]:
unique, counts = np.unique(labels, return_counts=True)
size_dict = dict(zip(unique, counts))
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

In [30]:
np.random.seed(0)

for i in range(len(sorted_clusters)):
  members = words[labels == sorted_clusters[i]]
  np.random.shuffle(members)
  print(' '.join(members[:50]))
  print('-'*80)

省略記号 リリイ・シュシュのすべて カムイ外伝 ")-( 2710 カガヤン・バレー 秋田1区 41.6% 日本翻訳大賞 615m 景仁 高度化PHS 水垣偉弥 ファラジ スターダストクルセイダース 中邨雄二 マイアサウラ 柳亭燕路 沢田ユキオ ウスバゼニゴケ 浮草 エルプロモーション ゲイラクス Evgeny オタリア スリーエム Yorktown ヴィダルサスーン 日本電気協会 カリュプソー ヨーロッパ卓球選手権 ロシアより愛をこめて 悪魔の手毬唄 ツチスドリ ニューロフィラメント 君田 トライガン 再生可能資源 ロー・ファンタジー YOSHI-HASHI 苛性カリ 荒雄川神社 裸名 コーパス言語学 デュアルヘッド Garnet Gartner シチリア海峡 宇土駅 ビスマルク海
--------------------------------------------------------------------------------
千葉県南房総市 東祖谷山村 盧竜県 田染 引地 サンマルク 白楽 東関東方言 五月山 ふきあげ 萩往還 多摩区 潮境 釜谷 鳥居本 赤城駅 うつくしま百名山 1567 荒沢 喜入町 石城郡 神呪寺 東京都東久留米市 新丸 元吉原 国民休暇村 木子 閩北 遠石 高崎町 幕張海浜公園 杉並町 伊勢赤十字病院 布引山地 道意 大森海岸駅 チャンタブリー県 是政 晋州市 阿寒郡 尾島町 多良間島 宮城県議会 西新井大師 862人 釜臥山 深安郡 新城町 浦内川 赤井川村
--------------------------------------------------------------------------------
スウィート・ホーム お笑い三人組 香南 マルグレ＝ヌー ジョージ・スコット ユルゲン・メルツァー NHK松江放送局 ボンダレンコ タレント議員 スウェーデングランプリ コングリッシュ ぱじゃまソフト マイケル・ハワード 大学路 クリントン国務長官 アルカリ剤 西沢歩 舁 ジロティ Al Jazeera 赤口 勧善懲悪委員会 ASEM 第24回東京国際映画祭 アボット・ラボラトリーズ イジメ、ダメ、ゼッタイ 基礎票 機動刑事ジバン エルサレム聖書 -5℃ 利根川水系8ダム ナルシソ・イエペス ファーストステップ

In [31]:
cluster_idx = sorted_clusters[0]
center = centers[cluster_idx]
indices = np.argsort(np.linalg.norm(U2 - center, axis=1))
filtered_indices = indices[labels[indices] == cluster_idx]
' '.join(words[filtered_indices[:100]].tolist())

'月刊ジェイ・ノベル referred 道哉 carried スペースコブラ こいずみまり 吉岡村 さすらい刑事旅情編 中村弘二 菊地創 内上 夜歩く 鈴木義尾 ラブ・アクチュアリー Kiss PLUS ラーマナータプラム 愛天使伝説ウェディングピーチ エーライツ 好夫 5850 ベストディガー Villanova 新潟県警察部 Tsuchiya 離婚弁護士 カガヤン・バレー ワンズファクトリー ビッグコミックオリジナル増刊 nikkei ")(" "）、" Asarum グッバイガール イースタン・パーク モンティ・パイソン・アンド・ホーリー・グレイル ブケディ シュレージシェ・オーバーラウジッツ ピースプロモーション おねがい☆ティーチャー ワーサル 月刊コミック@バンチ 押原 ダフラク オキソフラン シヴァガンガイ 招商銀行 春畑 アールグルッペ Missing Link ボーン・アイデンティティー 林由美香 浦和高等学校 Hiroyuki うたかたの恋 viridis 俺は用心棒 ドラゴンハート lg ペケ×ポン CEM 機動戦士ガンダム 閃光のハサウェイ 横戸 Eleganceイブ TRY AGAIN 慶南地方警察庁 ビッグコミック増刊号 西郷川 ドラゴンエイジ ソジ 「会いたかった」公演 サクラメリーメン SOLEIL ロード・オブ・ザ・リング/王の帰還 破裏拳ポリマー 古川学園 Nilsson ヨジテレビ! 蒼穹のファフナー edited 大森東 Kuraki 松本洋子 dimethyl ハイ!土曜日です とんがり帽子のメモル FCアムカル・ペルミ BLACK CAT 魔法にかけられて glucosidase ダックスープ ギンダラ かわみなみ Adriana はなればなれに きよせ くちよ ボウイ・ベイソックス セメナウィ・ケイバハリ 種子田健 センター・オブ・ジ・アース'

In [32]:
cluster_idx = sorted_clusters[99]
center = centers[cluster_idx]
indices = np.argsort(np.linalg.norm(U2 - center, axis=1))
filtered_indices = indices[labels[indices] == cluster_idx]
' '.join(words[filtered_indices[:100]].tolist())

'操ろ 破ろ 取り上げよ 借りよ 引こ 果たそ 尽くそ 押し付けよ 求めよ 立ち上がろ 持ち込も 忘れよ 壊そ 押さえ込も 当てよ 立て直そ かけよ 引き出そ よかろ 解こ 話そ 追いかけよ 向けよ 探ろ 受け入れよ 走ろ 取り込も 直そ 思い出そ 呼び戻そ 続けよ 掴も 確かめよ 選ぼ 呼ぼ 当たろ 送ろ 取り組も 見つけよ 働こ 作り出そ しまお 食い止めよ 開こ 言お まとめよ だそ 知らせよ 聞こ 創ろ 死の 試みよ 遅らせよ 上がろ 見極めよ 貰お 決めよ 追い出そ 騙そ 立ち去ろ とめよ 教えよ 乗り越えよ 扱お 歩も 組も 戻そ 加えよ 押さえよ 迫ろ 消そ 捕えよ 掛けよ 明かそ 取り入ろ 動かそ 持と 伝えよ 託そ 狙お 行なお 叩こ 返そ 乗っ取ろ 放と 励まそ 打と 代わろ 始めよ 救お 隠そ 探そ 加わろ 正そ 出そ 着せよ 鎮めよ 導こ 考えよ 込も'

In [33]:
cluster_idx = sorted_clusters[49]
center = centers[cluster_idx]
indices = np.argsort(np.linalg.norm(U2 - center, axis=1))
filtered_indices = indices[labels[indices] == cluster_idx]
' '.join(words[filtered_indices[:100]].tolist())

'小早川隆景 大内義弘 柴田勝家 佐々成政 里見義堯 毛利元就 相良氏 畠山重忠 北条氏直 上杉景勝 佐竹義重 毛利輝元 木曾義仲 元親 遺臣 豊臣氏 島津義久 長宗我部元親 秀長 細川勝元 源義仲 藤堂高虎 新田義貞 松永久秀 筒井順慶 武田勝頼 伊東氏 黒田孝高 最上義光 輝元 龍造寺隆信 今川義元 石田三成 大友宗麟 宇喜多直家 木曾義昌 尼子晴久 義元 晴信 真田昌幸 三好長慶 義武 羽柴秀吉 景春 結城合戦 昌幸 義弘 義宣 赤松氏 六角定頼 加賀一向一揆 蘆名 石山本願寺 龍造寺 宗瑞 安国寺恵瓊 小早川秀秋 荒木村重 長尾景虎 三好氏 北原氏 旧臣 長宗我部 足利持氏 和田義盛 為信 朝倉氏 今川 宇喜多秀家 総州家 佐東銀山城 氏綱 浦上宗景 宇喜多 寒河江氏 周防 顕家 尼子 旧主 三好三人衆 和田合戦 勝瑞城 佐久郡 目代 浅井氏 孝景 結城城 本山氏 足利義昭 太田道灌 長政 足利成氏 吉川元春 顕如 細川藤孝 北畠 筒井氏 三浦氏 明智光秀 別所氏'